In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline

In [6]:
investment_data = pd.read_csv('investment.csv')
tax_data = pd.read_csv('tax_rates.csv')
investment_data.head()

,Reference period,Business gross fixed capital formation,Residential structures,"Non-residential structures, machinery and equipment",Non-residential structures,Machinery and equipment,Intellectual property products
0,Q1 2000,212829.0,67962.0,117067.0,60702.0,56554.0,23888.0
1,Q2 2000,214884.0,68434.0,116805.0,59038.0,57352.0,25820.0
2,Q3 2000,217298.0,71454.0,115836.0,57418.0,57563.0,26906.0
3,Q4 2000,217890.0,72796.0,114254.0,57940.0,55979.0,28230.0
4,Q1 2001,222632.0,74929.0,115335.0,60415.0,55347.0,30010.0


In [14]:
# tax_data.head()
investment_data['Reference period'][0]

'Q1 2000'

## Data Manipulation

In [11]:
investment_data['Year'] = investment_data['Reference period'].apply(lambda s: s[3:])
investment_data.head()

TypeError: 'float' object is not subscriptable